# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
import time
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
env = gym.make('Breakout-v0')
#env.render()

In [3]:
number_lives = find_max_lifes(env)
state_size = env.observation_space.shape
action_size = env.action_space.n
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [4]:
agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0


### Main Training Loop

In [ ]:
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1
        if render_breakout:
            env.render()

        # Select and perform an action
        curr_state = history[3,:,:]
        action, value = agent.get_action(np.float32(history[:4, :, :]) / 255.)

        
        next_state, reward, done, info = env.step(action)

        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        #r = np.clip(reward, -1, 1)
        r = reward
        """
        if terminal_state:
            r -= 20
        """
        # Store the transition in memory 
        
        agent.memory.push(deepcopy(curr_state), action, r, terminal_state, value, 0, 0)
        # Start training after random sample generation
        if(frame % train_frame == 0):
            _, frame_next_val = agent.get_action(np.float32(history[1:, :, :]) / 255.)
            agent.train_policy_net(frame, frame_next_val)
            # Update the target network
            agent.update_target_net()
        score += r
        history[:4, :, :] = history[1:, :, :]

        if frame % 50000 == 0:
            print('now time : ', datetime.now())
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.savefig("./save_graph/breakout_dqn.png")

        if done:
            evaluation_reward.append(score)
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 10 episode is bigger than 400
            # stop training
            if np.mean(evaluation_reward) > 700 and len(evaluation_reward) > 40:
                torch.save(agent.policy_net, "./save_model/breakout_dqn")
                sys.exit()

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/michael/Documents/CS498DL/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))


episode: 0   score: 1.0   memory length: 236   epsilon: 1.0    steps: 236     evaluation reward: 1.0
episode: 1   score: 2.0   memory length: 532   epsilon: 1.0    steps: 296     evaluation reward: 1.5
episode: 2   score: 2.0   memory length: 806   epsilon: 1.0    steps: 274     evaluation reward: 1.6666666666666667
episode: 3   score: 0.0   memory length: 978   epsilon: 1.0    steps: 172     evaluation reward: 1.25
Training network. lr: 0.000250. clip: 0.100000


/home/michael/Documents/CS498DL/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:259: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  pol_loss += pol_avg.detach().cpu()[0]
/home/michael/Documents/CS498DL/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:260: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  vf_loss += value_loss.detach().cpu()[0]
/home/michael/Documents/CS498DL/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:261: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  ent_total += ent.detach().cpu()[0]


Iteration 1: Policy loss: 0.015309. Value loss: 0.022334. Entropy: 1.365773.
Iteration 2: Policy loss: 0.007594. Value loss: 0.022622. Entropy: 1.375099.
Iteration 3: Policy loss: 0.001630. Value loss: 0.021370. Entropy: 1.376557.
episode: 4   score: 1.0   memory length: 1024   epsilon: 1.0    steps: 211     evaluation reward: 1.2
episode: 5   score: 2.0   memory length: 1024   epsilon: 1.0    steps: 250     evaluation reward: 1.3333333333333333
episode: 6   score: 2.0   memory length: 1024   epsilon: 1.0    steps: 303     evaluation reward: 1.4285714285714286
episode: 7   score: 0.0   memory length: 1024   epsilon: 1.0    steps: 210     evaluation reward: 1.25
Training network. lr: 0.000250. clip: 0.099991
Iteration 4: Policy loss: 0.001487. Value loss: 0.022242. Entropy: 1.366803.
Iteration 5: Policy loss: 0.000655. Value loss: 0.022145. Entropy: 1.367186.
Iteration 6: Policy loss: 0.000348. Value loss: 0.022892. Entropy: 1.365525.
episode: 8   score: 1.0   memory length: 1024   epsi

episode: 49   score: 1.0   memory length: 1024   epsilon: 1.0    steps: 236     evaluation reward: 1.32
episode: 50   score: 2.0   memory length: 1024   epsilon: 1.0    steps: 280     evaluation reward: 1.3333333333333333
Training network. lr: 0.000250. clip: 0.099901
Iteration 34: Policy loss: -0.001704. Value loss: 0.019973. Entropy: 1.368357.
Iteration 35: Policy loss: -0.002906. Value loss: 0.018659. Entropy: 1.364466.
Iteration 36: Policy loss: -0.004754. Value loss: 0.017865. Entropy: 1.363726.
episode: 51   score: 0.0   memory length: 1024   epsilon: 1.0    steps: 182     evaluation reward: 1.3076923076923077
episode: 52   score: 1.0   memory length: 1024   epsilon: 1.0    steps: 201     evaluation reward: 1.3018867924528301
episode: 53   score: 0.0   memory length: 1024   epsilon: 1.0    steps: 169     evaluation reward: 1.2777777777777777
episode: 54   score: 0.0   memory length: 1024   epsilon: 1.0    steps: 170     evaluation reward: 1.2545454545454546
episode: 55   score: 0

...

episode: 4366   score: 4.0   memory length: 1024   epsilon: 1.0    steps: 355     evaluation reward: 3.87
Training network. lr: 0.000204. clip: 0.081577
Iteration 6142: Policy loss: 0.015903. Value loss: 0.007859. Entropy: 0.383651.
Iteration 6143: Policy loss: 0.008361. Value loss: 0.006722. Entropy: 0.382484.
Iteration 6144: Policy loss: 0.002929. Value loss: 0.006397. Entropy: 0.373023.
episode: 4367   score: 6.0   memory length: 1024   epsilon: 1.0    steps: 497     evaluation reward: 3.9
episode: 4368   score: 3.0   memory length: 1024   epsilon: 1.0    steps: 353     evaluation reward: 3.89
Training network. lr: 0.000204. clip: 0.081568
Iteration 6145: Policy loss: 0.005750. Value loss: 0.014065. Entropy: 0.369246.
Iteration 6146: Policy loss: -0.002063. Value loss: 0.010717. Entropy: 0.374226.
Iteration 6147: Policy loss: -0.008392. Value loss: 0.008733. Entropy: 0.382293.
episode: 4369   score: 5.0   memory length: 1024   epsilon: 1.0    steps: 462     evaluation reward: 3.91
e

episode: 4403   score: 6.0   memory length: 1024   epsilon: 1.0    steps: 513     evaluation reward: 3.94
Training network. lr: 0.000204. clip: 0.081442
Iteration 6187: Policy loss: 0.018224. Value loss: 0.008867. Entropy: 0.226253.
Iteration 6188: Policy loss: 0.001098. Value loss: 0.007034. Entropy: 0.239473.
Iteration 6189: Policy loss: -0.000211. Value loss: 0.006466. Entropy: 0.235732.
episode: 4404   score: 2.0   memory length: 1024   epsilon: 1.0    steps: 293     evaluation reward: 3.93
episode: 4405   score: 4.0   memory length: 1024   epsilon: 1.0    steps: 450     evaluation reward: 3.95
episode: 4406   score: 4.0   memory length: 1024   epsilon: 1.0    steps: 447     evaluation reward: 3.96
Training network. lr: 0.000204. clip: 0.081433
Iteration 6190: Policy loss: 0.016430. Value loss: 0.013498. Entropy: 0.351308.
Iteration 6191: Policy loss: -0.001178. Value loss: 0.010415. Entropy: 0.352530.
Iteration 6192: Policy loss: 0.000125. Value loss: 0.008578. Entropy: 0.361932.


episode: 4440   score: 2.0   memory length: 1024   epsilon: 1.0    steps: 285     evaluation reward: 3.9
episode: 4441   score: 3.0   memory length: 1024   epsilon: 1.0    steps: 355     evaluation reward: 3.9
episode: 4442   score: 2.0   memory length: 1024   epsilon: 1.0    steps: 317     evaluation reward: 3.85
Training network. lr: 0.000203. clip: 0.081307
Iteration 6232: Policy loss: 0.007500. Value loss: 0.014886. Entropy: 0.349011.
Iteration 6233: Policy loss: -0.001191. Value loss: 0.011934. Entropy: 0.359718.
Iteration 6234: Policy loss: -0.003911. Value loss: 0.010680. Entropy: 0.356230.
episode: 4443   score: 3.0   memory length: 1024   epsilon: 1.0    steps: 533     evaluation reward: 3.85
episode: 4444   score: 4.0   memory length: 1024   epsilon: 1.0    steps: 389     evaluation reward: 3.84
episode: 4445   score: 3.0   memory length: 1024   epsilon: 1.0    steps: 350     evaluation reward: 3.83
Training network. lr: 0.000203. clip: 0.081298
Iteration 6235: Policy loss: 0

Iteration 6274: Policy loss: 0.002906. Value loss: 0.007019. Entropy: 0.282886.
Iteration 6275: Policy loss: 0.001060. Value loss: 0.004801. Entropy: 0.294622.
Iteration 6276: Policy loss: -0.004072. Value loss: 0.004206. Entropy: 0.280899.
episode: 4479   score: 5.0   memory length: 1024   epsilon: 1.0    steps: 438     evaluation reward: 3.83
episode: 4480   score: 5.0   memory length: 1024   epsilon: 1.0    steps: 501     evaluation reward: 3.83
Training network. lr: 0.000203. clip: 0.081172
Iteration 6277: Policy loss: 0.016113. Value loss: 0.007262. Entropy: 0.226870.
Iteration 6278: Policy loss: 0.001680. Value loss: 0.005102. Entropy: 0.215612.
Iteration 6279: Policy loss: -0.008918. Value loss: 0.004499. Entropy: 0.218053.
episode: 4481   score: 4.0   memory length: 1024   epsilon: 1.0    steps: 428     evaluation reward: 3.82
episode: 4482   score: 5.0   memory length: 1024   epsilon: 1.0    steps: 450     evaluation reward: 3.84
episode: 4483   score: 3.0   memory length: 102

episode: 4516   score: 6.0   memory length: 1024   epsilon: 1.0    steps: 544     evaluation reward: 3.57
episode: 4517   score: 6.0   memory length: 1024   epsilon: 1.0    steps: 509     evaluation reward: 3.59
episode: 4518   score: 4.0   memory length: 1024   epsilon: 1.0    steps: 365     evaluation reward: 3.61
Training network. lr: 0.000203. clip: 0.081046
Iteration 6319: Policy loss: 0.009383. Value loss: 0.009285. Entropy: 0.294036.
Iteration 6320: Policy loss: 0.001861. Value loss: 0.008770. Entropy: 0.300908.
Iteration 6321: Policy loss: -0.005476. Value loss: 0.007693. Entropy: 0.293728.
episode: 4519   score: 4.0   memory length: 1024   epsilon: 1.0    steps: 382     evaluation reward: 3.6
episode: 4520   score: 4.0   memory length: 1024   epsilon: 1.0    steps: 438     evaluation reward: 3.61
Training network. lr: 0.000203. clip: 0.081037
Iteration 6322: Policy loss: 0.006909. Value loss: 0.009491. Entropy: 0.300893.
Iteration 6323: Policy loss: -0.008531. Value loss: 0.00

In [ ]:
torch.save(agent.policy_net, "./save_model/breakout_dqn")